<a href="https://colab.research.google.com/github/Ohara124c41/Frontier_AI-I/blob/main/What_is_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is PyTorch?

このノートブックでは，近年において最も主流な深層学習フレームワークのひとつであるPyTorchについて基本的な事項を説明し，以降で様々なモデルを実装していくための基礎を獲得することを目的とします．
# 目次

0.   準備
1.   テンソル生成方法
2.   種々の演算
3.   GPUの利用方法







In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 準備
**ファイルの設定**　本稿は Google Colabolatory 上で実行されることを想定して作成されています．
以降でGPUを用いた演算を解説する際の準備として，はじめにファイルの設定を変更します．

本稿を Google Colabolatory 上で開いたのち，上部メニューから「編集」→「ノートブックの設定」を選択し，ハードウェアアクセラレータをGPUに設定してください．この設定変更はそれまでに実行されていたランタイムを初期化するため，必ず最初に行うようにしてください．

以降，GPUによる演算が必要なノートブックでは適宜同様の設定を行なってください．

**PyTorchのインポート** Google Colabolatory ではPyTorchがあらかじめインストールされています．

In [8]:
import torch                   # PyTorchのインポート
print(torch.__version__)       # バージョン確認
print(torch.cuda.is_available()) # GPUが正しく利用できるかの確認
import numpy as np # numpyのインポート

2.0.1+cu118
True


## テンソルの生成方法
----
**サイズを指定したテンソル生成**

任意サイズのテンソルを生成するために，単純に領域を確保することができます．値の初期化はなされません．

以下では例としてベクトル・行列・高階テンソルの領域をそれぞれ確保しています．同様に，任意の自然数を列挙することで様々なサイズのテンソルを生成することができます．

In [9]:
# 1D vector
x = torch.empty(3)
print(x)

tensor([4.3427e-35, 0.0000e+00, 4.6526e-35])


In [10]:
# 2D matrix
x = torch.empty(2, 3)
print(x)

tensor([[4.6538e-35, 0.0000e+00, 4.6551e-35],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]])


In [11]:
# 3D tensor
x = torch.empty(2, 3, 2)
print(x)

tensor([[[ 0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00],
         [ 4.6553e-35,  0.0000e+00]],

        [[ 5.0383e-36,  0.0000e+00],
         [-1.5124e+29,  4.5818e-41],
         [ 0.0000e+00,  0.0000e+00]]])


生成したテンソルのサイズは以下のように確認することができます．

In [12]:
print(x.size())  # テンソルのサイズを取得
print(x.size(0)) # テンソルの0次元目のサイズを取得

torch.Size([2, 3, 2])
2


また，0や1などの特定の値や任意の値で埋めたテンソルを生成することができます．

In [13]:
# 0で埋める
x = torch.zeros(2, 2)
print(x)

tensor([[0., 0.],
        [0., 0.]])


In [14]:
# 1で埋める
x = torch.ones(2, 2)
print(x)

tensor([[1., 1.],
        [1., 1.]])


In [15]:
# 任意の値（ここでは3.1415）で埋める
x = torch.full((2, 2), 3.1415)
print(x)

tensor([[3.1415, 3.1415],
        [3.1415, 3.1415]])


乱数によって初期化されたテンソルを生成することもできます．

In [16]:
# 標準正規分布で初期化
x = torch.randn(2, 2)
print(x)

tensor([[ 1.4816, -1.1831],
        [-0.5222,  2.1324]])


In [17]:
# 0~1の一様分布で初期化
x = torch.rand(2, 2)
print(x)

tensor([[0.5087, 0.6844],
        [0.7508, 0.1236]])


----
**リストやnumpy.ndarrayからの変換**

あらかじめ定めた要素値によりテンソルを生成したい場合は，次のようにして実現できます．

In [18]:
# リストから生成
x = torch.tensor([[1., 2., 3.], [4., 5., 6.]])
print(x)

tensor([[1., 2., 3.],
        [4., 5., 6.]])


また，NumPy上でなんらかの計算を施した既存のndarrayをPyTorch上で利用したい場合も，同様のメソッドを用いることが可能です．このとき，ndarrayに設定されていた数値型がそのまま引き継がれます．

なお，print文による表示桁数は丸められてしまいますが，各要素には確かに数値型に対応する精度の数値が格納されています．

In [19]:
# 既存のndarrayとして準備
a = np.random.randn(3, 2)
print(a)
print(a.dtype)

[[ 0.41694637  0.31360781]
 [ 1.14116251 -1.28519308]
 [-0.49570746 -1.41118329]]
float64


In [20]:
# numpy.ndarrayから生成
x = torch.tensor(a)
print(x)

tensor([[ 0.4169,  0.3136],
        [ 1.1412, -1.2852],
        [-0.4957, -1.4112]], dtype=torch.float64)


----
**要素の取り出し**
リストやNumPyのndarrayと同様，スライスにより要素を取り出すことができます．

In [21]:
# 要素の取り出し
print(x[:2, :])
print(x[0, [1, 0]])

tensor([[ 0.4169,  0.3136],
        [ 1.1412, -1.2852]], dtype=torch.float64)
tensor([0.3136, 0.4169], dtype=torch.float64)


In [22]:
# 1要素のテンソルを単なるスカラー値に変換するために item を用いることができる
print(x[0, 0].item())

0.4169463719372803


----
**数値型**

PyTorchにもNumPyと同様に数値型の概念が存在します．数値型はテンソル生成時に指定できるほか，一度生成したテンソルを別の型に変換することもできます．

指定しない場合は基本的にfloat型とみなされます．よく用いられる型はfloat型およびlong型です．

In [23]:
# dtypeを指定することによるfloat型での生成（numpy.float32に対応）
x = torch.ones(2, 2, dtype=torch.float)
print(x)

tensor([[1., 1.],
        [1., 1.]])


既存のリストやnumpy.ndarrayを明示的にtorch.float型のテンソルへ変換することもできます．

In [24]:
# 先ほど用意した既存のndarray
print(a)
print(a.dtype)

[[ 0.41694637  0.31360781]
 [ 1.14116251 -1.28519308]
 [-0.49570746 -1.41118329]]
float64


In [25]:
# numpy.ndarrayからtorch.floatのテンソルを生成
x = torch.FloatTensor(a)
print(x)
print(x.dtype)

tensor([[ 0.4169,  0.3136],
        [ 1.1412, -1.2852],
        [-0.4957, -1.4112]])
torch.float32


In [26]:
# dtypeを指定することによるlong型での生成（numpy.int64に対応）
x = torch.ones(2, 2, dtype=torch.long)
print(x)

tensor([[1, 1],
        [1, 1]])


同様に，既存のリストやnumpy.ndarrayを明示的にtorch.long型のテンソルへ変換することができます．

In [27]:
# numpy.ndarrayからtorch.longのテンソルを生成
x = torch.LongTensor(a)
print(x)
print(x.dtype)

tensor([[ 0,  0],
        [ 1, -1],
        [ 0, -1]])
torch.int64


In [28]:
# torch.floatからtorch.longへの変換
f = torch.randn(4, 4)
print(f)

tensor([[ 1.0541,  1.0635,  0.7085, -1.0122],
        [ 0.4525, -0.8827,  1.7470,  0.9779],
        [ 0.0768, -0.9635, -0.2404,  0.6295],
        [-0.6970, -0.5757, -0.5014, -0.3381]])


In [29]:
l = f.long()
print(l)

tensor([[ 1,  1,  0, -1],
        [ 0,  0,  1,  0],
        [ 0,  0,  0,  0],
        [ 0,  0,  0,  0]])


In [30]:
# torch.longからtorch.floatへの変換
f2 = l.float()
print(f2)

tensor([[ 1.,  1.,  0., -1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]])


---
**既存のテンソルと同サイズのテンソル生成**

様々なアルゴリズムを実装していくにあたり，同サイズのテンソルを複数用意して演算を行うという場面は頻出します．そこでPyTorchには，あるテンソルを基準に同サイズのテンソルを簡単に生成できるようにするための機能が備わっています．

これにより，全てのテンソルに同じサイズを明示的に割り当てるコードを書く冗長性の排除や，テンソルサイズを可変にしてもコードの見通しが良くなるといった利点が生まれます．

In [31]:
# 既存テンソル
x = torch.empty(2, 5, 2)
print(x.size())

torch.Size([2, 5, 2])


In [32]:
# xと同サイズの0埋めテンソルを生成
y = torch.zeros_like(x)
print(y)

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])


In [33]:
# xと同サイズの1埋めテンソルを生成
y = torch.ones_like(x)
print(y)

tensor([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]])


In [34]:
# xと同サイズで任意の値で埋めたテンソルを生成
y = torch.full_like(x, 3.1415)
print(y)

tensor([[[3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415]],

        [[3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415],
         [3.1415, 3.1415]]])


In [35]:
# xと同サイズで標準正規分布で初期化されたテンソルを生成
y = torch.randn_like(x)
print(y)

tensor([[[ 0.0695, -0.7706],
         [-0.3700,  0.4894],
         [ 0.1749, -0.1102],
         [ 0.2061,  0.5465],
         [ 0.7068,  0.8311]],

        [[ 1.3112,  0.4898],
         [-2.1489, -0.4652],
         [ 0.3537,  1.0375],
         [ 1.5873,  2.0786],
         [-0.0347,  0.5039]]])


In [36]:
# xと同サイズで0~1の一様分布で初期化されたテンソルを生成
y = torch.rand_like(x)
print(y)

tensor([[[0.1848, 0.8811],
         [0.4913, 0.4113],
         [0.1918, 0.6234],
         [0.0534, 0.2755],
         [0.9015, 0.6984]],

        [[0.0720, 0.8044],
         [0.2971, 0.4188],
         [0.6909, 0.6690],
         [0.6167, 0.2208],
         [0.1012, 0.7863]]])


## 種々の演算
----
**四則演算**　基礎的な四則演算をテンソル同士で行うことができます．

In [37]:
# テンソルの準備
x = torch.randn(1, 2)
y = torch.randn(1, 2)
print(x)
print(y)

tensor([[-0.1633,  0.1147]])
tensor([[ 1.3723, -0.5154]])


In [38]:
# 加算
z = x + y
print(z)

tensor([[ 1.2090, -0.4007]])


In [39]:
# 減算
z = x - y
print(z)

tensor([[-1.5357,  0.6302]])


In [40]:
# 要素ごとの乗算
z = x * y
print(z)

tensor([[-0.2241, -0.0591]])


In [41]:
# 要素ごとの除算
z = x / y
print(z)

tensor([[-0.1190, -0.2226]])


In [42]:
# 冪乗
z = x**2
print(z)

tensor([[0.0267, 0.0132]])


---
**利用可能な数学関数** 様々な数学関数や集計関数が利用できます

In [43]:
x = torch.randn(1,2)
print(x)

tensor([[-1.6013, -1.7426]])


In [44]:
# 要素ごとの絶対値
z = torch.abs(x)
print(z)

tensor([[1.6013, 1.7426]])


In [45]:
# 要素ごとに三角関数を適用
z = torch.sin(x)
print(z)

tensor([[-0.9995, -0.9853]])


In [46]:
# 要素ごとに指数関数を適用
z = torch.exp(x)
print(z)

# 要素ごとに対数関数を適用
z = torch.log(x)
print(x)

tensor([[0.2016, 0.1751]])
tensor([[-1.6013, -1.7426]])


In [47]:
# 合計
z = torch.sum(x)
print(z)
# 最大値
z = torch.max(x)
print(z)
# 平均
z = torch.mean(x)
print(z)

tensor(-3.3439)
tensor(-1.6013)
tensor(-1.6719)


----
**テンソルサイズの変更**　
PyTorchのテンソルサイズはviewを用いて任意に整形できます．

In [48]:
x = torch.randn(2, 3, 4)
print(x)
print(x.size())

tensor([[[-1.5725,  0.3219, -1.1844,  0.7423],
         [-0.2702,  0.3092, -0.6425, -0.2409],
         [ 0.6757, -1.3511,  0.4156, -1.8324]],

        [[-0.5003,  0.2482,  0.5562,  1.0962],
         [ 1.7739, -0.1491, -0.2732, -1.1350],
         [-0.8487,  0.6569, -1.6491,  2.0404]]])
torch.Size([2, 3, 4])


In [49]:
y = x.view(6, 4)
print(y)
print(y.size())

tensor([[-1.5725,  0.3219, -1.1844,  0.7423],
        [-0.2702,  0.3092, -0.6425, -0.2409],
        [ 0.6757, -1.3511,  0.4156, -1.8324],
        [-0.5003,  0.2482,  0.5562,  1.0962],
        [ 1.7739, -0.1491, -0.2732, -1.1350],
        [-0.8487,  0.6569, -1.6491,  2.0404]])
torch.Size([6, 4])


In [50]:
# -1を指定すると整合するように自動で整数を推定してくれる
y = x.view(1, 2, -1, 2, 2)
print(y)
print(y.size())

tensor([[[[[-1.5725,  0.3219],
           [-1.1844,  0.7423]],

          [[-0.2702,  0.3092],
           [-0.6425, -0.2409]],

          [[ 0.6757, -1.3511],
           [ 0.4156, -1.8324]]],


         [[[-0.5003,  0.2482],
           [ 0.5562,  1.0962]],

          [[ 1.7739, -0.1491],
           [-0.2732, -1.1350]],

          [[-0.8487,  0.6569],
           [-1.6491,  2.0404]]]]])
torch.Size([1, 2, 3, 2, 2])


**軸の入れ替え**　テンソルにおける任意の2軸の順番を入れ替えることができます．

In [51]:
y = x.transpose(2, 0)
print(y)
print(y.size())

tensor([[[-1.5725, -0.5003],
         [-0.2702,  1.7739],
         [ 0.6757, -0.8487]],

        [[ 0.3219,  0.2482],
         [ 0.3092, -0.1491],
         [-1.3511,  0.6569]],

        [[-1.1844,  0.5562],
         [-0.6425, -0.2732],
         [ 0.4156, -1.6491]],

        [[ 0.7423,  1.0962],
         [-0.2409, -1.1350],
         [-1.8324,  2.0404]]])
torch.Size([4, 3, 2])


2次元行列の場合は単純に転置することも可能です．

In [52]:
m = torch.randn(3, 2)
print(m)
print(m.size())

tensor([[ 0.9446, -1.7339],
        [-0.4826,  0.1075],
        [-1.2095,  0.6240]])
torch.Size([3, 2])


In [53]:
# 転置行列を返す
n = m.t()
print(n)
print(n.size())

tensor([[ 0.9446, -0.4826, -1.2095],
        [-1.7339,  0.1075,  0.6240]])
torch.Size([2, 3])


全ての軸の順番を指定して入れ替えることも可能です．

In [54]:
# 軸の順列を指定して入れ替える
y = x.permute(2, 0, 1)
print(y)
print(y.size())

tensor([[[-1.5725, -0.2702,  0.6757],
         [-0.5003,  1.7739, -0.8487]],

        [[ 0.3219,  0.3092, -1.3511],
         [ 0.2482, -0.1491,  0.6569]],

        [[-1.1844, -0.6425,  0.4156],
         [ 0.5562, -0.2732, -1.6491]],

        [[ 0.7423, -0.2409, -1.8324],
         [ 1.0962, -1.1350,  2.0404]]])
torch.Size([4, 2, 3])


**メモリの整列**　上記にあげたテンソルの整形や軸の入れ替えは任意の操作数重ね合わせることが可能です．しかしながら，軸を入れ替えると表記上のテンソルと内部で確保されているメモリの並びが不整合となる場合があり，その後にviewメソッドを用いると失敗します．

In [55]:
# view size is not compatible with input tensor's size and stride... というエラーが生じる
# 今回はこの stride の部分に問題がある
z = x.transpose(2, 0).view(2, 2, 6)

RuntimeError: ignored

このようなエラーが発生した場合は，一度メモリの並びを整列させる操作を噛ませることで解消可能です．

In [56]:
z = x.transpose(2, 0).contiguous().view(2, 2, 6)
print(z)

tensor([[[-1.5725, -0.5003, -0.2702,  1.7739,  0.6757, -0.8487],
         [ 0.3219,  0.2482,  0.3092, -0.1491, -1.3511,  0.6569]],

        [[-1.1844,  0.5562, -0.6425, -0.2732,  0.4156, -1.6491],
         [ 0.7423,  1.0962, -0.2409, -1.1350, -1.8324,  2.0404]]])


----
**in-placeな演算**　
通常の演算では，出力テンソルとして新たなメモリ領域が確保されています．しかしながら，メモリの使用量を抑制したい場合等に，入力テンソルの要素をそのまま置換するように演算を行うことができます．

*   加算：add_
*   減算：sub_
*   乗算：mul_
*   除算：div_
*   転置：t_
*   軸の入れ替え：transpose_
*   値のコピー：copy_

等が用意されています．



In [57]:
# テンソルの準備
x = torch.randn(1, 2)
y = torch.randn(1, 2)
print(x)
print(y)

tensor([[0.3735, 1.0648]])
tensor([[-0.6685,  0.5345]])


In [58]:
# xにyをin-placeに加算する
x.add_(y)
# x自体の値が演算結果で書き換わる
print(x)

tensor([[-0.2950,  1.5993]])


In [59]:
# xをin-placeに転置する
x.t_()
print(x)

tensor([[-0.2950],
        [ 1.5993]])


In [60]:
# xの軸をin-placeに入れ替える
x.transpose_(0,1)
print(x)

tensor([[-0.2950,  1.5993]])


In [61]:
# xにyの値をコピーする
x.copy_(y)
print(x)

tensor([[-0.6685,  0.5345]])


ただし，後続のノートブックで説明する自動微分機能を用いる際は，基本的にin-placeな演算を行うことはできないことに留意してください．

## GPUの利用方法
----
**生成済のテンソルをGPUメモリに載せる**ことができます．複数枚のGPUが搭載されたサーバ上で実行する場合はGPU番号を指定することが推奨されますが，Google Colaboratory は1枚のみの搭載のため以下のようにして簡単に既存テンソルをGPU上のテンソル（CUDA Tensor）に変換することができます．

In [62]:
x_cuda = x.cuda()
print(x_cuda)

tensor([[-0.6685,  0.5345]], device='cuda:0')


テンソルの生成時にdeviceを指定することで，**初めからGPU上のメモリを確保してテンソルを生成する**ことも可能です．

In [63]:
x = torch.randn(1, 2, device="cuda:0")
print(x)

tensor([[-0.3361,  2.4687]], device='cuda:0')


存在しないGPU番号を指定するとエラーが発生します．

In [65]:
x = torch.randn(1, 2, device="cuda:1")

同じデバイス上のテンソルを用いて，先に紹介したような演算を同様に行うことが可能です．
ただし，異なるデバイス同士（CPUとGPU，または異なる番号のGPU同士）での演算はできません．

**CPUへの変換**　GPU上のテンソルは，同様に簡単にCPU上に移行することが可能です．

In [66]:
# CPUへ
x_cpu = x_cuda.cpu()
print(x_cpu)
print(x_cpu.device)

tensor([[-0.6685,  0.5345]])
cpu


**統一的な記法**　PyTorchが扱うテンソルは，デバイス名を文字列で指定することで簡単にデバイス間を移動させることができます．

In [67]:
# CPU上で生成
x = torch.randn(2, 2)
print(x.device)

cpu


In [68]:
# 0番GPUへ
x_cuda0 = x.to("cuda:0")
print(x_cuda0.device)

cuda:0


In [69]:
# CPUへ
x_cpu = x_cuda0.to("cpu")
print(x_cpu.device)

cpu


さらに，既存のテンソルを引数として，同じデバイス・同じ数値型への変換を行うことも可能です．

In [70]:
# 0番GPUにdouble型のテンソルを生成
d_cuda0 = torch.randn(2, 2, dtype=torch.double, device="cuda:0")
print(d_cuda0.device)
print(d_cuda0.dtype)

cuda:0
torch.float64


In [71]:
# xをd_cuda0と同じデバイス・数値型へ変換する
x_d = x.to(d_cuda0)
print(x_d.device)
print(x_d.dtype)

cuda:0
torch.float64


**他の配列オブジェクトへの変換**　CPU上のテンソルに限り，numpy.ndarrayやリストなどへ変換することができます．

In [72]:
# numpy.ndarrayに変換
x_ndarray = x_cpu.numpy()
print(x_ndarray)

[[ 1.8329135   0.09140043]
 [ 2.650712   -0.62957746]]


In [73]:
# リストに変換
x_list = x_cpu.tolist()
print(x_list)

[[1.8329135179519653, 0.09140042960643768], [2.650712013244629, -0.6295774579048157]]


In [74]:
x = np.random.randn(3,2)
y = np.random.randn(3,2)
print(x)
print(y)
z=(x - y)**2
print(z)
s = np.sum(z, axis=1)
print(s)
m = np.mean(s)
print(m)

[[-1.01432509 -0.51834784]
 [ 0.36264149  0.71662493]
 [-0.44086812  0.59520184]]
[[-0.55094914 -0.03862722]
 [-0.91697527 -0.54245321]
 [ 0.08843144  0.23879624]]
[[0.21471727 0.23013187]
 [1.63741905 1.58527775]
 [0.28015803 0.12702495]]
[0.44484914 3.2226968  0.40718298]
1.3582429728085128


# 課題
<h4>
平均二乗誤差関数の実装
</h4>

 
入力：$\boldsymbol{X}=(\boldsymbol{x}_1,\boldsymbol{x}_2,\cdots\boldsymbol{x}_{N})^{T}\in\mathbb{R}^{N\times M},　\boldsymbol{Y}=(\boldsymbol{y}_1,\boldsymbol{y}_2,\cdots\boldsymbol{y}_{N})^{T}\in\mathbb{R}^{N\times M}$
      
出力：$z\in\mathbb{R}, z=L(\boldsymbol{X}, \boldsymbol{Y}~)$
    
    

- 平均二乗誤差 :
$$ 
L(\boldsymbol{X},\boldsymbol{Y}~) = \frac{1}{N}\sum^N_i \|\boldsymbol{x}_i-\boldsymbol{y}_i\|_2^2\\
$$
Numpyのndarrayを受け取り，GPU上で平均二乗誤差を計算して，GPU上のメモリに乗ったテンソルを出力する関数を実装してみましょう


In [76]:
def MSE(x, y):
  # TODO
  x = torch.tensor(x).cuda()
  y = torch.tensor(y).cuda()
  z = torch.mean(torch.sum((x-y)**2, dim=1))

  return z

In [ ]:
from google.colab import drive # driveを接続
drive.mount('/content/gdrive')

In [75]:
# drive中の課題ファイルのあるディレクトリに移動
%cd /content/drive/MyDrive/__FrontierAI-I/FAI20230606/data
from pytorchtest import *

/content/drive/MyDrive/__FrontierAI-I/FAI20230606/data


In [77]:
test_MSE(MSE)

OK


OKと出力されれば完成です

最終項で紹介したGPU上での演算は極めて重要であり，今日では専らGPUを用いて演算することが主流となっています．
というのも，一般の配列操作や行列演算等はGPUを用いることで高速に並列計算可能なためです．このような装置の恩恵もあって今日の深層学習モデルは現実的な時間で学習させることが可能となっています．

----
お疲れ様でした．本稿は以上で終了いたします．
以降のハンズオンでは、深層学習において重要な概念のひとつである自動微分機能について解説したのち，具体的なデータセットとタスクを通してより深層学習モデルに関する知見を深めていきます．